
## **PromptTemplate**

* A core LangChain abstraction for structuring prompts.
* Lets you define a **template string** with placeholders (like `{product}`), then fill them dynamically at runtime.
* Useful because LLMs respond better to structured, consistent prompts instead of messy string concatenation.
* Example:

  ```python
  from langchain.prompts import PromptTemplate

  prompt = PromptTemplate(
      input_variables=["product"],
      template="Write a tagline for {product}."
  )
  print(prompt.format(product="smartwatch"))
  ```
* In real systems, PromptTemplate is critical for **standardizing prompts across pipelines**—for chatbots, knowledge assistants, or content generation tools.

---

## **Runnable (LCEL) vs LangGraph**

1. **Runnable / LCEL (LangChain Expression Language)**

   * LCEL makes LangChain components composable, like building blocks in a pipeline.
   * Each step is a **Runnable** (prompt → model → parser).
   * Encourages functional chaining: data flows through transforms.
   * Example: `prompt | llm | parser`
   * Ideal when you want **linear flows**—like passing user query → LLM → JSON output.
   * Industry use: building **ETL-like LLM workflows** (parse documents, enrich, then output summaries).

2. **LangGraph**

   * Built on top of LCEL but for **graphs instead of chains**.
   * Lets you design **state machines / DAGs (directed acyclic graphs)** where control flow depends on conditions.
   * Can implement loops, branching, memory, retries.
   * Example: Customer support bot where path depends on sentiment: positive → FAQ answer; negative → escalate to human.
   * Industry use: **multi-agent orchestration** or complex **decision trees with LLMs**.

---

Simple metaphor:

* **LCEL** = a train track, straight line, predictable stops.
* **LangGraph** = a metro map, multiple routes, branches, loops, and junctions.
 


---
---

- Langserver
- Langsmith


## Memory: 
## sessions(message History):
## hub & agent executor:
## tools (all including yt tool, serach tool, math tools, embeeding tools, indexing tools ):
## sql toolkit vs mcp : 
## stuff document chain text summarization vs map reduce summarization technique with single prompt and multiple prompt template vs refine chain summarization:
## huggingfaceXlangchain features:
##  



# LangChain Complete Guide: Level 1-3

## 🎯 Level 1: Fundamentals

### 1.1 What is LangChain?
Framework for building LLM-powered applications with composable components.

### 1.2 Installation
```bash
pip install langchain langchain-openai langchain-community
pip install python-dotenv  # for API keys
```

### 1.3 Basic Setup
```python
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = "your-api-key"

# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
```

### 1.4 Simple Chat
```python
from langchain.schema import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="What is Python?")
]

response = llm.invoke(messages)
print(response.content)
```

### 1.5 Prompt Templates
```python
from langchain.prompts import PromptTemplate

# Basic template
template = "Tell me a {adjective} joke about {topic}"
prompt = PromptTemplate(template=template, input_variables=["adjective", "topic"])

# Generate prompt
formatted = prompt.format(adjective="funny", topic="cats")
print(formatted)
```

### 1.6 Chat Prompt Templates
```python
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a {role}"),
    ("human", "{user_input}")
])

messages = chat_template.format_messages(
    role="travel guide",
    user_input="Best places in Japan?"
)

response = llm.invoke(messages)
```

### 1.7 Simple Chains (LCEL)
```python
from langchain_core.output_parsers import StrOutputParser

# Chain: Prompt → LLM → Output Parser
chain = prompt | llm | StrOutputParser()

result = chain.invoke({"adjective": "silly", "topic": "programmers"})
print(result)
```

---

## 🚀 Level 2: Intermediate

### 2.1 Output Parsers

**String Parser**
```python
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = prompt | llm | parser
```

**JSON Parser**
```python
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline")

parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Tell a joke.\n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | llm | parser
result = chain.invoke({})
```

### 2.2 Memory

**Conversation Buffer Memory**
```python
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "Hi!"}, {"output": "Hello! How can I help?"})

print(memory.load_memory_variables({}))
```

**Conversation with Memory**
```python
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

print(conversation.predict(input="Hi, I'm Alice"))
print(conversation.predict(input="What's my name?"))
```

**Window Memory (Last N messages)**
```python
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)  # Keep last 2 interactions
```

### 2.3 Document Loading & Processing

**Load Documents**
```python
from langchain_community.document_loaders import TextLoader, PyPDFLoader

# Text file
loader = TextLoader("document.txt")
docs = loader.load()

# PDF
pdf_loader = PyPDFLoader("document.pdf")
pages = pdf_loader.load_and_split()
```

**Text Splitting**
```python
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

chunks = text_splitter.split_documents(docs)
```

### 2.4 Embeddings & Vector Stores

**Create Embeddings**
```python
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Initialize embeddings
embeddings = OpenAIEmbeddings()

# Create vector store
vectorstore = FAISS.from_documents(chunks, embeddings)

# Save/Load
vectorstore.save_local("faiss_index")
vectorstore = FAISS.load_local("faiss_index", embeddings)
```

**Similarity Search**
```python
query = "What is the main topic?"
results = vectorstore.similarity_search(query, k=3)

for doc in results:
    print(doc.page_content)
```

### 2.5 Retrieval QA

**Basic RAG**
```python
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

answer = qa_chain.invoke("What is the document about?")
print(answer['result'])
```

**With Custom Prompt**
```python
from langchain.prompts import PromptTemplate

template = """Use the following context to answer the question.
If you don't know, say "I don't know".

Context: {context}

Question: {question}

Answer:"""

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT}
)
```

### 2.6 Agents (Basic)

**Simple Agent**
```python
from langchain.agents import load_tools, initialize_agent, AgentType

# Load tools
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Initialize agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run
result = agent.run("What is the population of Tokyo in 2023?")
```

---

## 🔥 Level 3: Advanced

### 3.1 Custom Tools

**Create Custom Tool**
```python
from langchain.tools import Tool
from langchain.agents import initialize_agent

def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [
    Tool(
        name="Word Length",
        func=get_word_length,
        description="Useful for getting the length of a word"
    )
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)
```

**Using @tool Decorator**
```python
from langchain.tools import tool

@tool
def search_api(query: str) -> str:
    """Search for information using an API."""
    # Your API logic here
    return f"Results for: {query}"

tools = [search_api]
```

### 3.2 Custom Chains

**Sequential Chain**
```python
from langchain.chains import SequentialChain, LLMChain

# Chain 1: Generate synopsis
synopsis_prompt = PromptTemplate(
    input_variables=["title"],
    template="Write a synopsis for a movie titled '{title}'"
)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt, output_key="synopsis")

# Chain 2: Generate review
review_prompt = PromptTemplate(
    input_variables=["synopsis"],
    template="Write a review based on this synopsis:\n{synopsis}"
)
review_chain = LLMChain(llm=llm, prompt=review_prompt, output_key="review")

# Combine
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["title"],
    output_variables=["synopsis", "review"]
)

result = overall_chain({"title": "The AI Revolution"})
```

**Router Chain**
```python
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

# Define specialized prompts
physics_template = """You are a physics expert. Answer: {input}"""
math_template = """You are a math expert. Answer: {input}"""

prompt_infos = [
    {"name": "physics", "description": "Good for physics questions", "prompt_template": physics_template},
    {"name": "math", "description": "Good for math questions", "prompt_template": math_template}
]

# Create router
destination_chains = {}
for p_info in prompt_infos:
    prompt = PromptTemplate(template=p_info['prompt_template'], input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[p_info["name"]] = chain

router_chain = MultiPromptChain(...)  # Simplified
```

### 3.3 Conversational RAG

**With Chat History**
```python
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

chat_history = []

# First question
result = qa({"question": "What is LangChain?", "chat_history": chat_history})
chat_history.append((result['question'], result['answer']))

# Follow-up
result = qa({"question": "Can you elaborate?", "chat_history": chat_history})
```

### 3.4 Multi-Query Retrieval

```python
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm
)

# Generates multiple queries for better retrieval
docs = retriever.get_relevant_documents("What is machine learning?")
```

### 3.5 Streaming Responses

```python
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0.7,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chain = prompt | llm | StrOutputParser()
for chunk in chain.stream({"topic": "AI"}):
    print(chunk, end="", flush=True)
```

### 3.6 Custom Retriever

```python
from langchain.schema import BaseRetriever, Document

class CustomRetriever(BaseRetriever):
    documents: list[Document]
    
    def get_relevant_documents(self, query: str) -> list[Document]:
        # Custom retrieval logic
        return [doc for doc in self.documents if query.lower() in doc.page_content.lower()]
    
    async def aget_relevant_documents(self, query: str) -> list[Document]:
        return self.get_relevant_documents(query)

retriever = CustomRetriever(documents=chunks)
```

### 3.7 Complex Agent with Multiple Tools

```python
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Setup tools
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [
    wikipedia,
    search_api,  # Your custom tool
]

# Create agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "Tell me about LangChain and search for latest news"})
```

### 3.8 Self-Querying Retriever

```python
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(name="source", description="The source document", type="string"),
    AttributeInfo(name="page", description="The page number", type="integer"),
]

retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Research papers on AI",
    metadata_field_info=metadata_field_info
)

docs = retriever.get_relevant_documents("Papers from 2023 about GPT")
```

---

## 📚 Key Concepts Summary

### LCEL (LangChain Expression Language)
```python
# Chain components with |
chain = prompt | llm | parser

# Parallel execution with RunnableParallel
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    joke=joke_chain,
    poem=poem_chain
)
```

### Callbacks
```python
from langchain.callbacks import StdOutCallbackHandler

chain.invoke(input_data, config={"callbacks": [StdOutCallbackHandler()]})
```

### Caching
```python
from langchain.cache import InMemoryCache
import langchain

langchain.llm_cache = InMemoryCache()
```

---

## 🎓 Best Practices

1. **Always handle errors** with try-except blocks
2. **Use environment variables** for API keys
3. **Chunk documents appropriately** (1000-2000 chars)
4. **Add metadata** to documents for better retrieval
5. **Use streaming** for better UX with long responses
6. **Monitor token usage** to control costs
7. **Test prompts** iteratively for best results
8. **Use appropriate memory** type for your use case

---

## 🔗 Common Patterns

**RAG Pipeline**
```
Load → Split → Embed → Store → Retrieve → Generate
```

**Agent Flow**
```
Input → Reasoning → Tool Selection → Tool Execution → Response
```

**Chain Types**
- `stuff`: Put all docs in prompt (simple, token-limited)
- `map_reduce`: Summarize each doc, then combine
- `refine`: Iteratively refine answer with each doc
- `map_rerank`: Score each doc's answer, return best

---

This guide covers essential LangChain concepts from basics to advanced patterns. Practice each level before moving to the next!

---
---
---
---

# LangGraph Complete Guide: Level 1-3

## 🎯 Level 1: Fundamentals

### 1.1 What is LangGraph?
Framework for building **stateful, multi-actor applications** with LLMs using **graphs**. Think of it as state machines for AI agents.

**Key Concepts:**
- **Nodes**: Functions that process state
- **Edges**: Connections between nodes
- **State**: Shared data structure
- **Graph**: The workflow definition

### 1.2 Installation
```bash
pip install langgraph langchain langchain-openai
```

### 1.3 Basic Setup
```python
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "your-api-key"
llm = ChatOpenAI(model="gpt-4")
```

### 1.4 Define State
```python
class State(TypedDict):
    messages: list[str]
    counter: int
```

### 1.5 Simple Graph - Hello World
```python
from langgraph.graph import StateGraph, END

# Define state
class State(TypedDict):
    message: str

# Define node function
def say_hello(state: State) -> State:
    return {"message": "Hello, World!"}

# Build graph
workflow = StateGraph(State)
workflow.add_node("hello", say_hello)
workflow.set_entry_point("hello")
workflow.add_edge("hello", END)

# Compile and run
app = workflow.compile()
result = app.invoke({"message": ""})
print(result)  # {'message': 'Hello, World!'}
```

### 1.6 Linear Chain
```python
class State(TypedDict):
    input: str
    output: str

def node_1(state: State) -> State:
    return {"output": f"Processed: {state['input']}"}

def node_2(state: State) -> State:
    return {"output": f"{state['output']} -> Enhanced"}

# Build
workflow = StateGraph(State)
workflow.add_node("process", node_1)
workflow.add_node("enhance", node_2)

workflow.set_entry_point("process")
workflow.add_edge("process", "enhance")
workflow.add_edge("enhance", END)

app = workflow.compile()
result = app.invoke({"input": "Hello"})
print(result['output'])  # Processed: Hello -> Enhanced
```

### 1.7 Conditional Edges
```python
class State(TypedDict):
    number: int
    result: str

def check_number(state: State) -> State:
    return state

def positive_path(state: State) -> State:
    return {"result": "Number is positive"}

def negative_path(state: State) -> State:
    return {"result": "Number is negative"}

# Conditional routing function
def route_number(state: State) -> str:
    if state["number"] > 0:
        return "positive"
    else:
        return "negative"

# Build graph
workflow = StateGraph(State)
workflow.add_node("check", check_number)
workflow.add_node("positive", positive_path)
workflow.add_node("negative", negative_path)

workflow.set_entry_point("check")
workflow.add_conditional_edges(
    "check",
    route_number,
    {
        "positive": "positive",
        "negative": "negative"
    }
)
workflow.add_edge("positive", END)
workflow.add_edge("negative", END)

app = workflow.compile()
print(app.invoke({"number": 5}))   # positive
print(app.invoke({"number": -3}))  # negative
```

---

## 🚀 Level 2: Intermediate

### 2.1 Chat Agent with LLM
```python
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import Annotated
from langgraph.graph.message import add_messages

# State with message reducer
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: AgentState) -> AgentState:
    return {"messages": [llm.invoke(state["messages"])]}

# Build
workflow = StateGraph(AgentState)
workflow.add_node("agent", chatbot)
workflow.set_entry_point("agent")
workflow.add_edge("agent", END)

app = workflow.compile()

# Use
result = app.invoke({
    "messages": [HumanMessage(content="Tell me a joke")]
})
print(result["messages"][-1].content)
```

### 2.2 Memory with Checkpointing
```python
from langgraph.checkpoint.memory import MemorySaver

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Use with thread_id for conversation
config = {"configurable": {"thread_id": "conversation-1"}}

result1 = app.invoke({
    "messages": [HumanMessage(content="My name is Alice")]
}, config)

result2 = app.invoke({
    "messages": [HumanMessage(content="What's my name?")]
}, config)

print(result2["messages"][-1].content)  # Should remember "Alice"
```

### 2.3 Agent with Tools
```python
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

tools = [multiply, add]
tool_node = ToolNode(tools)

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

def agent(state: AgentState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

def should_continue(state: AgentState) -> str:
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return "end"

# Build graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end": END
    }
)
workflow.add_edge("tools", "agent")

app = workflow.compile()

# Use
result = app.invoke({
    "messages": [HumanMessage(content="What is 5 multiplied by 3?")]
})
print(result["messages"][-1].content)
```

### 2.4 Human-in-the-Loop
```python
from langgraph.graph import END

class State(TypedDict):
    messages: Annotated[list, add_messages]
    approval: str

def agent_node(state: State):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

def human_review(state: State):
    # This will pause and wait for human input
    return state

def check_approval(state: State) -> str:
    if state.get("approval") == "approved":
        return "continue"
    return "end"

workflow = StateGraph(State)
workflow.add_node("agent", agent_node)
workflow.add_node("human", human_review)

workflow.set_entry_point("agent")
workflow.add_edge("agent", "human")
workflow.add_conditional_edges(
    "human",
    check_approval,
    {
        "continue": "agent",
        "end": END
    }
)

app = workflow.compile(checkpointer=MemorySaver(), interrupt_before=["human"])

# Use
config = {"configurable": {"thread_id": "1"}}
result = app.invoke({"messages": [HumanMessage("Generate a report")]}, config)

# Later, after review
app.update_state(config, {"approval": "approved"})
result = app.invoke(None, config)
```

### 2.5 Subgraphs
```python
# Create a subgraph
def create_research_graph():
    class ResearchState(TypedDict):
        query: str
        results: list[str]
    
    def search(state: ResearchState):
        return {"results": [f"Result for {state['query']}"]}
    
    sub_workflow = StateGraph(ResearchState)
    sub_workflow.add_node("search", search)
    sub_workflow.set_entry_point("search")
    sub_workflow.add_edge("search", END)
    
    return sub_workflow.compile()

# Use in main graph
research_graph = create_research_graph()

class MainState(TypedDict):
    topic: str
    research_data: list[str]

def research_node(state: MainState):
    result = research_graph.invoke({"query": state["topic"]})
    return {"research_data": result["results"]}

workflow = StateGraph(MainState)
workflow.add_node("research", research_node)
workflow.set_entry_point("research")
workflow.add_edge("research", END)

app = workflow.compile()
```

### 2.6 Parallel Execution
```python
from langgraph.graph import StateGraph

class State(TypedDict):
    input: str
    output_a: str
    output_b: str
    final: str

def task_a(state: State):
    return {"output_a": f"Task A: {state['input']}"}

def task_b(state: State):
    return {"output_b": f"Task B: {state['input']}"}

def combine(state: State):
    return {"final": f"{state['output_a']} + {state['output_b']}"}

workflow = StateGraph(State)
workflow.add_node("task_a", task_a)
workflow.add_node("task_b", task_b)
workflow.add_node("combine", combine)

workflow.set_entry_point("task_a")
workflow.set_entry_point("task_b")  # Both start simultaneously
workflow.add_edge("task_a", "combine")
workflow.add_edge("task_b", "combine")
workflow.add_edge("combine", END)

app = workflow.compile()
```

---

## 🔥 Level 3: Advanced

### 3.1 Multi-Agent System
```python
class MultiAgentState(TypedDict):
    messages: Annotated[list, add_messages]
    next_agent: str
    task_complete: bool

# Define specialist agents
researcher_llm = llm.bind_tools([]).with_structured_output({"research": "string"})
writer_llm = llm.bind_tools([]).with_structured_output({"draft": "string"})
critic_llm = llm.bind_tools([]).with_structured_output({"feedback": "string"})

def researcher(state: MultiAgentState):
    prompt = f"Research this topic: {state['messages'][-1].content}"
    result = llm.invoke([HumanMessage(content=prompt)])
    return {
        "messages": [AIMessage(content=result.content, name="Researcher")],
        "next_agent": "writer"
    }

def writer(state: MultiAgentState):
    research = state["messages"][-1].content
    prompt = f"Write an article based on: {research}"
    result = llm.invoke([HumanMessage(content=prompt)])
    return {
        "messages": [AIMessage(content=result.content, name="Writer")],
        "next_agent": "critic"
    }

def critic(state: MultiAgentState):
    draft = state["messages"][-1].content
    prompt = f"Critique this draft: {draft}"
    result = llm.invoke([HumanMessage(content=prompt)])
    
    # Simplified: assume always approved
    return {
        "messages": [AIMessage(content=result.content, name="Critic")],
        "next_agent": "end",
        "task_complete": True
    }

def router(state: MultiAgentState) -> str:
    return state.get("next_agent", "end")

# Build
workflow = StateGraph(MultiAgentState)
workflow.add_node("researcher", researcher)
workflow.add_node("writer", writer)
workflow.add_node("critic", critic)

workflow.set_entry_point("researcher")
workflow.add_conditional_edges(
    "researcher",
    router,
    {"writer": "writer", "end": END}
)
workflow.add_conditional_edges(
    "writer",
    router,
    {"critic": "critic", "end": END}
)
workflow.add_conditional_edges(
    "critic",
    router,
    {"writer": "writer", "end": END}
)

app = workflow.compile()
```

### 3.2 Self-Reflection Loop
```python
class ReflectionState(TypedDict):
    messages: Annotated[list, add_messages]
    iterations: int
    max_iterations: int
    is_good: bool

def generate(state: ReflectionState):
    prompt = state["messages"][-1].content if state["messages"] else "Write a story"
    result = llm.invoke([HumanMessage(content=prompt)])
    return {
        "messages": [AIMessage(content=result.content, name="Generator")],
        "iterations": state.get("iterations", 0) + 1
    }

def reflect(state: ReflectionState):
    content = state["messages"][-1].content
    prompt = f"Critique and improve this: {content}"
    result = llm.invoke([HumanMessage(content=prompt)])
    
    # Simple quality check
    is_good = "excellent" in result.content.lower() or state["iterations"] >= state["max_iterations"]
    
    return {
        "messages": [AIMessage(content=result.content, name="Reflector")],
        "is_good": is_good
    }

def should_continue(state: ReflectionState) -> str:
    if state.get("is_good", False):
        return "end"
    if state.get("iterations", 0) >= state.get("max_iterations", 3):
        return "end"
    return "continue"

workflow = StateGraph(ReflectionState)
workflow.add_node("generate", generate)
workflow.add_node("reflect", reflect)

workflow.set_entry_point("generate")
workflow.add_edge("generate", "reflect")
workflow.add_conditional_edges(
    "reflect",
    should_continue,
    {
        "continue": "generate",
        "end": END
    }
)

app = workflow.compile()

result = app.invoke({
    "messages": [HumanMessage("Write a short story about AI")],
    "max_iterations": 3
})
```

### 3.3 Planning and Execution Agent
```python
class PlanExecuteState(TypedDict):
    input: str
    plan: list[str]
    past_steps: list[tuple[str, str]]
    current_step: int
    final_response: str

def planner(state: PlanExecuteState):
    prompt = f"Create a step-by-step plan for: {state['input']}"
    result = llm.invoke([HumanMessage(content=prompt)])
    
    # Parse plan (simplified)
    plan = [line.strip() for line in result.content.split('\n') if line.strip()]
    
    return {
        "plan": plan,
        "current_step": 0
    }

def executor(state: PlanExecuteState):
    current_step = state["current_step"]
    step = state["plan"][current_step]
    
    prompt = f"Execute this step: {step}\nContext: {state['past_steps']}"
    result = llm.invoke([HumanMessage(content=prompt)])
    
    past_steps = state.get("past_steps", [])
    past_steps.append((step, result.content))
    
    return {
        "past_steps": past_steps,
        "current_step": current_step + 1
    }

def should_continue(state: PlanExecuteState) -> str:
    if state["current_step"] >= len(state["plan"]):
        return "finalize"
    return "execute"

def finalizer(state: PlanExecuteState):
    prompt = f"Summarize the results: {state['past_steps']}"
    result = llm.invoke([HumanMessage(content=prompt)])
    return {"final_response": result.content}

workflow = StateGraph(PlanExecuteState)
workflow.add_node("planner", planner)
workflow.add_node("executor", executor)
workflow.add_node("finalizer", finalizer)

workflow.set_entry_point("planner")
workflow.add_edge("planner", "executor")
workflow.add_conditional_edges(
    "executor",
    should_continue,
    {
        "execute": "executor",
        "finalize": "finalizer"
    }
)
workflow.add_edge("finalizer", END)

app = workflow.compile()
```

### 3.4 Dynamic Routing with Context
```python
from typing import Literal

class RouterState(TypedDict):
    messages: Annotated[list, add_messages]
    intent: str
    data: dict

def classifier(state: RouterState):
    content = state["messages"][-1].content
    
    # Use LLM to classify intent
    prompt = f"Classify this request into: 'search', 'calculate', or 'chat': {content}"
    result = llm.invoke([HumanMessage(content=prompt)])
    
    intent = "chat"  # default
    if "search" in result.content.lower():
        intent = "search"
    elif "calculate" in result.content.lower():
        intent = "calculate"
    
    return {"intent": intent}

def search_handler(state: RouterState):
    # Simulate search
    return {"messages": [AIMessage(content="Search results...")]}

def calculate_handler(state: RouterState):
    return {"messages": [AIMessage(content="Calculation result...")]}

def chat_handler(state: RouterState):
    result = llm.invoke(state["messages"])
    return {"messages": [result]}

def route_intent(state: RouterState) -> str:
    return state.get("intent", "chat")

workflow = StateGraph(RouterState)
workflow.add_node("classifier", classifier)
workflow.add_node("search", search_handler)
workflow.add_node("calculate", calculate_handler)
workflow.add_node("chat", chat_handler)

workflow.set_entry_point("classifier")
workflow.add_conditional_edges(
    "classifier",
    route_intent,
    {
        "search": "search",
        "calculate": "calculate",
        "chat": "chat"
    }
)
workflow.add_edge("search", END)
workflow.add_edge("calculate", END)
workflow.add_edge("chat", END)

app = workflow.compile()
```

### 3.5 Streaming Responses
```python
from langgraph.graph import StateGraph

# Streaming events
async def stream_example():
    async for event in app.astream({"messages": [HumanMessage("Hello")]}):
        print(event)

# Streaming tokens
async def stream_tokens():
    async for chunk in app.astream_log({"messages": [HumanMessage("Tell me a story")]}):
        print(chunk)

# Synchronous streaming
for event in app.stream({"messages": [HumanMessage("Hello")]}, stream_mode="values"):
    print(event)
```

### 3.6 Time Travel / State Replay
```python
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

# Execute multiple steps
app.invoke({"messages": [HumanMessage("Step 1")]}, config)
app.invoke({"messages": [HumanMessage("Step 2")]}, config)
app.invoke({"messages": [HumanMessage("Step 3")]}, config)

# Get all checkpoints
checkpoints = list(app.get_state_history(config))

# Go back to previous state
for checkpoint in checkpoints:
    print(f"Step: {checkpoint.values}")
    
# Resume from a specific checkpoint
app.invoke(None, checkpoints[1].config)
```

### 3.7 Custom Checkpointer (Database)
```python
from langgraph.checkpoint.base import BaseCheckpointSaver
from typing import Optional

class DatabaseCheckpointer(BaseCheckpointSaver):
    def __init__(self, db_connection):
        self.db = db_connection
        super().__init__()
    
    def put(self, config, checkpoint, metadata):
        # Save to database
        thread_id = config["configurable"]["thread_id"]
        # self.db.save(thread_id, checkpoint, metadata)
        pass
    
    def get(self, config):
        # Load from database
        thread_id = config["configurable"]["thread_id"]
        # return self.db.load(thread_id)
        pass

# Usage
# db_checkpoint = DatabaseCheckpointer(your_db)
# app = workflow.compile(checkpointer=db_checkpoint)
```

### 3.8 Hierarchical Agent Teams
```python
class TeamState(TypedDict):
    messages: Annotated[list, add_messages]
    team: str
    task_results: dict

# Team 1: Research Team
def create_research_team():
    def researcher(state):
        return {"messages": [AIMessage(content="Research done", name="Researcher")]}
    
    def analyst(state):
        return {"messages": [AIMessage(content="Analysis done", name="Analyst")]}
    
    workflow = StateGraph(TeamState)
    workflow.add_node("researcher", researcher)
    workflow.add_node("analyst", analyst)
    workflow.set_entry_point("researcher")
    workflow.add_edge("researcher", "analyst")
    workflow.add_edge("analyst", END)
    
    return workflow.compile()

# Team 2: Writing Team
def create_writing_team():
    def writer(state):
        return {"messages": [AIMessage(content="Draft written", name="Writer")]}
    
    def editor(state):
        return {"messages": [AIMessage(content="Edited", name="Editor")]}
    
    workflow = StateGraph(TeamState)
    workflow.add_node("writer", writer)
    workflow.add_node("editor", editor)
    workflow.set_entry_point("writer")
    workflow.add_edge("writer", "editor")
    workflow.add_edge("editor", END)
    
    return workflow.compile()

# Supervisor
research_team = create_research_team()
writing_team = create_writing_team()

class SupervisorState(TypedDict):
    messages: Annotated[list, add_messages]
    next_team: str

def supervisor(state: SupervisorState):
    # Decide which team to use
    prompt = f"Which team should handle: {state['messages'][-1].content}? (research/writing)"
    result = llm.invoke([HumanMessage(content=prompt)])
    
    if "research" in result.content.lower():
        return {"next_team": "research"}
    return {"next_team": "writing"}

def research_node(state):
    result = research_team.invoke(state)
    return {"messages": result["messages"]}

def writing_node(state):
    result = writing_team.invoke(state)
    return {"messages": result["messages"]}

def route_team(state: SupervisorState) -> str:
    return state.get("next_team", "end")

workflow = StateGraph(SupervisorState)
workflow.add_node("supervisor", supervisor)
workflow.add_node("research_team", research_node)
workflow.add_node("writing_team", writing_node)

workflow.set_entry_point("supervisor")
workflow.add_conditional_edges(
    "supervisor",
    route_team,
    {
        "research": "research_team",
        "writing": "writing_team",
        "end": END
    }
)
workflow.add_edge("research_team", END)
workflow.add_edge("writing_team", END)

app = workflow.compile()
```

---

## 📊 Key Patterns

### State Management
```python
# Reducer functions for state updates
from operator import add

class State(TypedDict):
    # Append to list
    messages: Annotated[list, add_messages]
    
    # Replace value
    counter: int
    
    # Custom reducer
    scores: Annotated[list[int], add]
```

### Error Handling
```python
def safe_node(state: State):
    try:
        # Your logic
        return {"result": "success"}
    except Exception as e:
        return {"error": str(e), "result": "failed"}
```

### Conditional Logic
```python
def router(state: State) -> Literal["path_a", "path_b", "end"]:
    if state["condition"]:
        return "path_a"
    elif state["other_condition"]:
        return "path_b"
    return "end"
```

---

## 🎓 Best Practices

1. **State Design**: Keep state minimal and clear
2. **Type Hints**: Always use TypedDict for state
3. **Node Functions**: Keep them pure and focused
4. **Error Handling**: Wrap nodes in try-except
5. **Checkpointing**: Use for production deployments
6. **Testing**: Test individual nodes before graph
7. **Visualization**: Use `app.get_graph().draw_mermaid()` to visualize
8. **Memory**: Choose appropriate checkpointer (Memory/SQLite/Postgres)

---

## 🔧 Debugging

### Visualize Graph
```python
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

# Or as Mermaid text
print(app.get_graph().draw_mermaid())
```

### Inspect State
```python
config = {"configurable": {"thread_id": "1"}}
current_state = app.get_state(config)
print(current_state.values)
print(current_state.next)  # Next node to execute
```

### Debug Mode
```python
# Add debug info to nodes
def debug_node(state: State):
    print(f"Current state: {state}")
    result = process(state)
    print(f"Output: {result}")
    return result
```

---

## 🚀 Common Use Cases

1. **Chatbot with Memory**: Simple conversation agent
2. **RAG Agent**: Retrieval + Generation with tools
3. **Multi-Agent Research**: Specialized agents collaborate
4. **Plan-Execute**: Break down and execute complex tasks
5. **Human-in-Loop**: Review and approve agent actions
6. **Reflection**: Self-improving outputs
7. **Hierarchical Teams**: Manager + worker agents
8. **State Machine**: Complex workflows with conditions

---

## 📝 Quick Reference

```python
# Basic Structure
workflow = StateGraph(State)
workflow.add_node("name", function)
workflow.set_entry_point("name")
workflow.add_edge("from", "to")
workflow.add_conditional_edges("from", router, {"path": "node"})
app = workflow.compile()

# With Memory
app = workflow.compile(checkpointer=MemorySaver())

# Invoke
result = app.invoke(input, config={"configurable": {"thread_id": "1"}})

# Stream
for event in app.stream(input):
    print(event)
```

---

This guide covers LangGraph from basics to advanced patterns. Master Level 1 before moving to complex multi-agent systems! 🎯